In [1]:
import pandas as pd
import numpy as np
import os
import ddf_utils.ddf_reader as ddf

In [2]:
ddf.SEARCH_PATH = "/Users/semio/src/work/Gapminder/"

In [3]:
l = pd.read_csv('./GW_indicators_list.csv')

In [4]:
l2 = l[['gw_ddf_id', 'found_in_ddf', 'ddf_concept_id', 'Notes']]
l2 = l2[~l2['found_in_ddf'].isnull()]

In [6]:
geo = ddf.ddf_entities('ddf--gapminder--geo_entity_domain')['country']

In [7]:
def get_geo_map(geo_2, column='area'):

    geo_map = {} 

    cols = ['alternative_1', 'alternative_2', 'alternative_3', 'gapminder_list',
           'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name',
           'code', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6', 'name']

    for i, v in geo_2.iterrows():

        geo_map[v[column]] = np.nan


        for c in cols:
            f = geo[geo[c] == v['name'].strip()]
            if len(f) > 0:
                geo_map[v[column]] = f['country'].values[0]
            if len(f) > 1:
                print(f)
                
    return geo_map

In [8]:
# OECD

In [10]:
oecd = l2[l2['found_in_ddf'].str.contains('oecd')]

oecd

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
128,aid_given_2007_us,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
129,aid_given_per_person_2007_us,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
130,aid_given_percent_of_gni,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
131,economical_infrastructure_aid_given_percent_of...,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
132,education_aid_given_percent_of_aid,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
133,government_and_society_aid_given_percent_of_aid,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
134,health_aid_given_percent_of_aid,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
135,multisector_cross_cutting_aid_given_percent_of...,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
136,other_social_services_aid_given_percent_of_aid,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....
137,population_policies_aid_given_percent_of_aid,ddf--oecd--dac,NaN,1. OECD don't have 2007 dollar data already 2....


In [24]:
geo2 = ddf.ddf_entities('ddf--oecd--dac')['donor']

geo_map = get_geo_map(geo2, 'donor')

In [25]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('donor').ix[a.index]

,name
104,Nordic Development Fund [NDF]
552,Kuwait [KFAED]
811,Global Environment Facility [GEF]
901,International Bank for Reconstruction and Deve...
903,International Finance Corporation [IFC]
905,International Development Association [IDA]
906,Caribbean Development Bank [CarDB]
909,Inter-American Development Bank [IDB]
912,IDB Special Fund
913,African Development Bank [AfDB]


In [242]:
name = oecd['gw_ddf_id'].values[11]
name_2 = 'total_oda'

sector_id = 140

In [230]:
name

'water_and_sanitation_aid_given_percent_of_aid'

In [123]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [232]:
right = ddf.ddf_datapoint('ddf--oecd--dac', name_2)

In [233]:
right = right.query("donor not in @a.index & sector == @sector_id")[['donor', 'year', name_2]]

In [234]:
right.columns = ['geo', 'time', name]

right.geo = right.geo.map(lambda x: geo_map[x])

In [235]:
right = right.set_index(['geo', 'time'])

In [236]:
total = ddf.ddf_datapoint('ddf--oecd--dac', name_2).query("sector == 1000")

In [237]:
total.donor = total.donor.map(lambda x: geo_map[x])

In [238]:
to_concat = []

for c in right.index.levels[0]:
    total_ = total[total.donor == c].set_index('year')
    t  = total_['total_oda'].copy()
    
    t.index.name = 'time'
    
    r = right.ix[c][name]
    
    assert np.all(r.duplicated()) == False
    assert np.all(t.duplicated()) == False
    
    f = r / t * 100
    
    f = f.reset_index()
    f['geo'] = c
    
    to_concat.append(f)

In [239]:
final = pd.concat(to_concat)

final.columns = ['time', name, 'geo']

In [240]:
final.sort_values(by=['geo', 'time']).dropna(how='any')[['geo', name, 'time']]\
.to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)